# Part 1

### Setup to read from Wikipedia

In [1]:
pip install wikipedia

     |████████████████████████████████| 112kB 16.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

     |████████████████████████████████| 5.8MB 21.0MB/s eta 0:00:01��██████████▋         | 4.1MB 21.0MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [70]:
import wikipedia
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### Read table into a dataframe

In [41]:
page = wikipedia.page("List of postal codes of Canada: M").html().encode("UTF-8")
toronto_df = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])
toronto_df = pd.read_html(page)[0]
toronto_df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


### Remove all rows where Borough = Not assigned

In [42]:
toronto_df.drop(toronto_df[toronto_df["Borough"]=="Not assigned"].index, inplace=True)
toronto_df.reset_index(drop=True, inplace=True)
toronto_df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


### Assigning Borough value to Neighborhood where Neighborhood = Not assigned

In [43]:
for row in toronto_df.index:
    if toronto_df.iloc[row, 2] == "Not assigned":
        toronto_df.iloc[row, 2] = toronto_df.iloc[row, 1]
toronto_df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


### Combining Neighborhood column for rows of same Postal Code

In [44]:
toronto_df_final = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])
toronto_df_final['PostalCode'] = toronto_df["Postcode"]
toronto_df_final['Borough'] = toronto_df["Borough"]
toronto_df_final['Neighborhood'] = ""

for row in toronto_df_final.index:
    for rows in toronto_df.index:
        if toronto_df_final.iloc[row, 0] == toronto_df.iloc[rows, 0]:
            toronto_df_final.iloc[row, 2] = toronto_df.iloc[rows, 2] + ', ' + toronto_df_final.iloc[row, 2]

toronto_df_final.drop_duplicates(subset=['PostalCode'], inplace = True)
toronto_df_final.reset_index(drop=True, inplace=True)
toronto_df_final

,PostalCode,Borough,Neighborhood
0,M3A,North York,"Parkwoods,"
1,M4A,North York,"Victoria Village,"
2,M5A,Downtown Toronto,"Harbourfront,"
3,M6A,North York,"Lawrence Manor, Lawrence Heights,"
4,M7A,Downtown Toronto,"Queen's Park,"
...,...,...,...
98,M8X,Etobicoke,"Old Mill North, Montgomery Road, The Kingsway,"
99,M4Y,Downtown Toronto,"Church and Wellesley,"
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 East...
101,M8Y,Etobicoke,"Sunnylea, Royal York South East, The Queensway..."


### Obtain dataframe shape

In [45]:
toronto_df_final.shape

(103, 3)

# Part 2

### Obtaining the coordinates of each postal code

In [46]:
lat_long = pd.read_csv('Geospatial_Coordinates.csv')
lat_long.sort_values(by='Postal Code', inplace=True)

toronto_df_final.sort_values(by='PostalCode', inplace=True)
toronto_df_final["Latitude"] = lat_long['Latitude']
toronto_df_final["Longitude"] = lat_long['Longitude']

toronto_df_final.reset_index(drop=True, inplace=True)
toronto_df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge,",43.727929,-79.262029
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek,",43.794200,-79.262029
2,M1E,Scarborough,"West Hill, Morningside, Guildwood,",43.778517,-79.346556
3,M1G,Scarborough,"Woburn,",43.770120,-79.408493
4,M1H,Scarborough,"Cedarbrae,",43.745906,-79.352188
...,...,...,...,...,...
98,M9N,York,"Weston,",43.696948,-79.411307
99,M9P,Etobicoke,"Westmount,",43.648429,-79.382280
100,M9R,Etobicoke,"St. Phillips, Richview Gardens, Martin Grove G...",43.647927,-79.419750
101,M9V,Etobicoke,"Thistletown, South Steeles, Silverstone, Mount...",43.602414,-79.543484


# Part 3

### Creating the map

In [59]:
import folium

toronto_coord = [43.6532, -79.3832]

toronto_map = folium.Map(location=toronto_coord, zoom_start=10.5)

for lat, long, postalcode, borough in zip(toronto_df_final['Latitude'], toronto_df_final['Longitude'], toronto_df_final['PostalCode'], toronto_df_final['Borough']):
    label = '{}, {}'.format(postalcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(toronto_map)
    
toronto_map

### I will explore the Downtown borough only

In [100]:
# Obtaining Postal Codes & Coordinates for Downtown Toronto

downtown_codes = toronto_df_final.loc[toronto_df_final["Borough"] == 'Downtown Toronto']
downtown_codes.drop(['Borough', 'Neighborhood'], axis = 1, inplace = True)
downtown_codes.reset_index(drop = True, inplace = True)
downtown_codes.rename(columns={'PostalCode': 'Postal Code'}, inplace=True)
downtown_codes

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,Postal Code,Latitude,Longitude
0,M4W,43.636258,-79.498509
1,M4X,43.756303,-79.565963
2,M4Y,43.696319,-79.532242
3,M5A,43.763573,-79.188711
4,M5B,43.692657,-79.264848
5,M5C,43.799525,-79.318389
6,M5E,43.757490,-79.374714
7,M5G,43.782736,-79.442259
8,M5H,43.737473,-79.464763
9,M5J,43.695344,-79.318389


### Setting up Foursquare

In [49]:
import requests

CLIENT_ID = 'UEHS0BUJQOPZQPPKWNBA1YZSCP3O14SZYGT1ZC0XECDCJFMG'
CLIENT_SECRET = '3AM1HHTVARNZV5VMLJKY25YQOXKYHC0M3R45IZL3QQ4LXB5S'
VERSION = '20180604'
LIMIT = 1
radius = 500
lat = downtown_codes.iloc[0, 1] # an example
long = downtown_codes.iloc[0, 2] # an example

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat,
    long,
    radius,
    LIMIT)

### Loop through all Downtown postal codes and explore them

In [153]:
postal_codes = downtown_codes["Postal Code"]
latitudes = downtown_codes["Latitude"]
longitudes = downtown_codes["Longitude"]
radius = 500
LIMIT = 50

venues = []

for postal_code, lat, long in zip(postal_codes, latitudes, longitudes):

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        long, 
        radius, 
        LIMIT)
            
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    venues.append([(
        postal_code, 
        lat, 
        long, 
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])

In [154]:
nearby_venues = pd.DataFrame(columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category'])

for i in range(0, len(venues)):
    nearby_venues.loc[i] = [venues[i][0][0]] + [venues[i][0][1]] + [venues[i][0][2]] + [venues[i][0][3]] + [venues[i][0][4]] + [venues[i][0][5]] + [venues[i][0][6]]
    
nearby_venues.head()

,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4W,43.636258,-79.498509,Eggsmart,43.638400,-79.499430,Breakfast Spot
1,M4X,43.756303,-79.565963,Pizza Monza,43.755043,-79.567195,Pizza Place
2,M4Y,43.696319,-79.532242,Starbucks,43.696405,-79.533479,Coffee Shop
3,M5A,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,M5B,43.692657,-79.264848,The Birchcliff,43.691666,-79.264532,Café


In [52]:
nearby_venues_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

nearby_venues_onehot['Postal Code'] = nearby_venues['Postal Code'] 

fixed_columns = [nearby_venues_onehot.columns[-1]] + list(nearby_venues_onehot.columns[:-1])
nearby_venues_onehot = nearby_venues_onehot[fixed_columns]

nearby_venues_onehot.head()

,Postal Code,Airport,Baseball Field,Breakfast Spot,Brewery,Burger Joint,Cafeteria,Café,Chinese Restaurant,Coffee Shop,Food,Gym,Hakka Restaurant,Park,Pizza Place,Sandwich Place,Skating Rink
0,M4W,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M4X,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,M4Y,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,M5B,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [103]:
venues_grouped = nearby_venues_onehot.groupby('Postal Code').mean().reset_index()
venues_grouped.head()

,Postal Code,Airport,Baseball Field,Breakfast Spot,Brewery,Burger Joint,Cafeteria,Café,Chinese Restaurant,Coffee Shop,Food,Gym,Hakka Restaurant,Park,Pizza Place,Sandwich Place,Skating Rink
0,M4W,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M4X,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,M4Y,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,M5B,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [54]:
num_top_venues = 5

for hood in venues_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = venues_grouped[venues_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4W----
            venue  freq
0  Breakfast Spot   1.0
1         Airport   0.0
2  Baseball Field   0.0
3         Brewery   0.0
4    Burger Joint   0.0


----M4X----
            venue  freq
0     Pizza Place   1.0
1         Airport   0.0
2  Baseball Field   0.0
3  Breakfast Spot   0.0
4         Brewery   0.0


----M4Y----
            venue  freq
0     Coffee Shop   1.0
1         Airport   0.0
2  Baseball Field   0.0
3  Breakfast Spot   0.0
4         Brewery   0.0


----M5A----
            venue  freq
0     Pizza Place   1.0
1         Airport   0.0
2  Baseball Field   0.0
3  Breakfast Spot   0.0
4         Brewery   0.0


----M5B----
            venue  freq
0            Café   1.0
1         Airport   0.0
2  Baseball Field   0.0
3  Breakfast Spot   0.0
4         Brewery   0.0


----M5C----
                venue  freq
0  Chinese Restaurant   1.0
1             Airport   0.0
2      Baseball Field   0.0
3      Breakfast Spot   0.0
4             Brewery   0.0


----M5E----
            venu

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [96]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = venues_grouped['Postal Code']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Breakfast Spot,Skating Rink,Sandwich Place,Pizza Place,Park,Hakka Restaurant,Gym,Food,Coffee Shop,Chinese Restaurant
1,M4X,Pizza Place,Skating Rink,Sandwich Place,Park,Hakka Restaurant,Gym,Food,Coffee Shop,Chinese Restaurant,Café
2,M4Y,Coffee Shop,Skating Rink,Sandwich Place,Pizza Place,Park,Hakka Restaurant,Gym,Food,Chinese Restaurant,Café
3,M5A,Pizza Place,Skating Rink,Sandwich Place,Park,Hakka Restaurant,Gym,Food,Coffee Shop,Chinese Restaurant,Café
4,M5B,Café,Skating Rink,Sandwich Place,Pizza Place,Park,Hakka Restaurant,Gym,Food,Coffee Shop,Chinese Restaurant


In [134]:
kclusters = 7

venues_grouped_clustering = venues_grouped.drop('Postal Code', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)

kmeans.labels_[0:10] 

neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

venues_merged = downtown_codes
venues_merged = venues_merged.join(neighborhoods_venues_sorted.set_index('Postal Code'), on='Postal Code')

map_clusters = folium.Map(location=toronto_coord, zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(venues_merged['Latitude'], venues_merged['Longitude'], venues_merged['Postal Code'], venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [137]:
venues_merged.loc[venues_merged['Cluster Labels'] == 0, venues_merged.columns[0:5]]

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue
0,M4W,43.636258,-79.498509,0,Breakfast Spot
2,M4Y,43.696319,-79.532242,0,Coffee Shop
5,M5C,43.799525,-79.318389,0,Chinese Restaurant
6,M5E,43.757490,-79.374714,0,Cafeteria
9,M5J,43.695344,-79.318389,0,Skating Rink
12,M5S,43.691116,-79.476013,0,Sandwich Place
13,M5T,43.651571,-79.484450,0,Food
15,M5W,43.628841,-79.520999,0,Burger Joint
17,M6G,43.753259,-79.329656,0,Park
18,M7A,43.773136,-79.239476,0,Hakka Restaurant


In [138]:
venues_merged.loc[venues_merged['Cluster Labels'] == 1, venues_merged.columns[0:5]]

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue
10,M5K,43.668999,-79.315572,1,Gym
11,M5L,43.689574,-79.383160,1,Gym


In [129]:
venues_merged.loc[venues_merged['Cluster Labels'] == 3, venues_merged.columns[0:5]]

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M5X,43.724766,-79.532242,2,Baseball Field,Skating Rink,Sandwich Place,Pizza Place,Park,Hakka Restaurant,Gym,Food,Coffee Shop,Chinese Restaurant


In [130]:
venues_merged.loc[venues_merged['Cluster Labels'] == 4, venues_merged.columns[0:5]]

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5B,43.692657,-79.264848,3,Café,Skating Rink,Sandwich Place,Pizza Place,Park,Hakka Restaurant,Gym,Food,Coffee Shop,Chinese Restaurant


In [131]:
venues_merged.loc[venues_merged['Cluster Labels'] == 5, venues_merged.columns[0:5]]

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4X,43.756303,-79.565963,4,Pizza Place,Skating Rink,Sandwich Place,Park,Hakka Restaurant,Gym,Food,Coffee Shop,Chinese Restaurant,Café
3,M5A,43.763573,-79.188711,4,Pizza Place,Skating Rink,Sandwich Place,Park,Hakka Restaurant,Gym,Food,Coffee Shop,Chinese Restaurant,Café
7,M5G,43.782736,-79.442259,4,Pizza Place,Skating Rink,Sandwich Place,Park,Hakka Restaurant,Gym,Food,Coffee Shop,Chinese Restaurant,Café


In [132]:
venues_merged.loc[venues_merged['Cluster Labels'] == 6, venues_merged.columns[0:5]]

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M5H,43.737473,-79.464763,5,Airport,Skating Rink,Sandwich Place,Pizza Place,Park,Hakka Restaurant,Gym,Food,Coffee Shop,Chinese Restaurant


In [133]:
venues_merged.loc[venues_merged['Cluster Labels'] == 7, venues_merged.columns[0:5]]

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M5V,43.662744,-79.321558,6,Brewery,Skating Rink,Sandwich Place,Pizza Place,Park,Hakka Restaurant,Gym,Food,Coffee Shop,Chinese Restaurant


## I performed the same analysis as the lab, except on a subset of the city (I choose Downtonwn Toronto only). The first impression is that the dataset was not large enough to have significant clustering (only 19 rows). Analysing the entire city might have yielded more relevant results